# ROB2004 Final Project - pick and place objects

The goal of this project is to solve a simple manipulation task: create a tower of cubes.

## Instructions
* Solve the problem below.
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a report (maximum 3 pages) in pdf format detailing the methodology you followed to solve the problem. You should add the plots in the report (does not count for the page limit).
* This is an individual project

We will use a model of the Frank-Emika Panda robot. This robot has 7 revolute joints and its kinematics is described in the picture below.

You will need to install pybullet on your computer. Please reach out as soon as possible for help if you do not know how to do it.

<div>
<img src="./panda.jpg" width="200"/>
</div>

# Problem

Implement controller(s) using the code skeletong below to get the robot to build a tower of 3 cubes, with the green cube on the bottom and the blue cube on the top. 

<div>
<img src="./manipulation_scene.jpg" width="400"/>
</div>

You are free to use the methods that you want to solve the task, with the following constraints:
* You cannot use any external library apart from numpy and scipy
* You need to use at least one controller in the end-effector space
* You need to use at least one inverse geometry method
* You need to compensate for the gravity of the robot

In your report, describe the controller(s) and justify your choices. Analyze the behavior of the system (include plots of the end-effector trajectories, velocities, joint trajectories, etc as you see fit).

## Scene details
The position of the red cube in the spatial frame is $(0.35,0.58,0.65)$, the green cube is at $(0.2,0.58,0.65)$ and the blue cube at $(0.0,0.58,0.65)$.


In [1]:
import numpy as np
import time

import matplotlib.pyplot as plt

from panda import Simulator, PandaRobot
from scipy import optimize

np.set_printoptions(precision=4, suppress = True)

# Helper functions

We provide a set of helper functions (forward kinematics, Jacobians, gravity terms) that can be used to implement the desired controllers. These functions are implemented in the PandaRobot class (in the panda.py file).
Examples are shown below:

In [2]:
# create an instance of the helper class
my_robot = PandaRobot()

# compute the forward kinematics for an arbitrary joint configuration
q = np.array([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])
pose = my_robot.FK(q)
print(f'the pose of the end-effector for joint configuration\n\n {q}\n\n is\n\n {pose}')

the pose of the end-effector for joint configuration

 [ 0.      0.      0.     -1.5708  0.      1.5708  0.    ]

 is

 [[-0.7071  0.7071  0.      0.    ]
 [ 0.7071  0.7071 -0.      0.5545]
 [-0.      0.     -1.      0.6245]
 [ 0.      0.      0.      1.    ]]


In [3]:
# we can also get the Jacobian in various frames 
# it is sufficient to pass the name of the frame as an argument as shown below

q = np.random.uniform(-1,1,7)

spatial_jacobian = my_robot.get_jacobian(q, 'S')
body_jacobian = my_robot.get_jacobian(q, 'B')

# we can even get the Jacobian in a frame place at the same position as the end-effector 
# frame but oriented like the spatial frame
oriented_jacobian = my_robot.get_jacobian(q, 'O')

In [4]:
# finally it is possible to compute the effect of gravity on the joints
# this can for example be used to do gravity compensation when controlling the robot

q = np.random.uniform(-1,1,7)

g = my_robot.g(q)

print(f'for joint configuration \n\n {q} \n \n the gravity force seen on the joints is \n \n {g}')

for joint configuration 

 [ 0.3114  0.8947  0.7987 -0.2451 -0.0019 -0.3782 -0.2765] 
 
 the gravity force seen on the joints is 
 
 [  0.     -47.3276   4.507   12.8845   0.3585  -0.3914   0.    ]


In [5]:
def trajectory(position_init, position_goal, t_init, t_goal, t):
    T = t_goal - t_init
    s = 3/(T**2)*((t-t_init)**2) - 2/ (T**3)*(t-t_init)**3
    desired_position = position_init + s * (position_goal - position_init)
    desired_velocity = (6/(T**2)*(t-t_init) - 6/(T**3)*(t-t_init)**2) * (position_goal - position_init)
    return desired_position, desired_velocity

def end_effector_position_error(joint_angles, desired_end_effector_position):
    """
    end_effector_position_error function
    input joint_angles (as a 3D vector) and desired_end_effector_position (as a 3D vector)
    output the squared distance between the desired end_effector position and the one realized by joint_angles
    """
    p_fk = my_robot.FK(joint_angles)
    error = np.subtract(p_fk[:3, 3], desired_end_effector_position)
    return error.dot(error)


def inverse_geometry(desired_end_effector_position):
    """
    inverse geometry function
    input desired_end_effector_position (3D vector of desired position of the end_effector)
    output a vector of joint angles
    """
    res = optimize.minimize(end_effector_position_error, np.zeros(7), args = (desired_end_effector_position))
    return res.x


# Code Skeleton for the simulation

Feel free to change the `run_time` variable to match your needs. The class `Simulator` contains all the code for the simulation. The code skeleton currently does not do anything.

Importantly you can control both the joints and the gripper of the robot:
* `simulator.send_joint_torque(joint_torques)` sends a vector of dimension 7 setting the torques of the joints
* `simulator.gripper_move(gripper_position)` sends a vector of dimension 2 setting the position of the fingers of the gripper. The fingers can have positions from 0 to 0.04cm (you can fully close the gripper by setting `simulator.gripper_move([0,0])` and fully open it by setting `simulator.gripper_move([0.04,0.04)`)

In [6]:
# here we create a simulation and reset the state of the robot
simulator = Simulator()
simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0])
simulator.gripper_move([0.04,0.04])

my_robot = PandaRobot()



# we store information
ndofs = 7 # number of degrees of freedom (excluding the gripper)
#The position of the red cube in the spatial frame is $(0.35,0.58,0.65)$, the green cube is at $(0.2,0.58,0.65)$ and the blue cube at $(0.0,0.58,0.65)$
red_cube = np.array([0.35,0.58,0.65])
green_cube = np.array([0.2,0.58,0.65])
blue_cube = np.array([0.0,0.58,0.65])
desired_position = np.array([0,0,0])
desired_velocity = np.array([0,0,0])
time_to_goal = 0.75


above_red = red_cube
red_cube = red_cube - np.array([0.004, -0.04, 0.34])
above_blue = blue_cube
blue_cube = blue_cube - np.array([0.00355,-0.0795, 0.402+0.062])
above_green = green_cube
green_cube = green_cube - np.array([0.0035,-0.05,0.34])
top_stack = green_cube - np.array([0.004,0,-0.1])
goals = [above_red, red_cube, [0,0], red_cube, above_green, green_cube, [0.04,0.04], above_green, blue_cube, [0,0], blue_cube, above_green, top_stack, [0.04,0.04], above_green]
end_effector_starting_pose = my_robot.FK(simulator.get_state()[0])[:3, 3]


run_time = time_to_goal*len(goals)
# simulation time step
dt = 0.005
num_steps = int(run_time/dt)
time = np.zeros([num_steps])

K_P = np.array([1.0, 1.0, 1.0]) * 5
K_D = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])*30

previous_goal = [0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0.0]
goal = inverse_geometry(goals[0])

### this is the main control loop
for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q, dq = simulator.get_state()
    pose = my_robot.FK(q)
    end_effector_position = pose[:3,3]
    
    goal_index = int(time[i]//time_to_goal)
    if np.array(goals[goal_index]).shape == (2,):
        simulator.gripper_move(goals[goal_index])
        continue
    else:
        previous_goal = goal
        goal = inverse_geometry(goals[goal_index])
    start_position = previous_goal
    start_time = time_to_goal*goal_index
    end_time = start_time + time_to_goal
    trajectory_position, trajectory_velocity = trajectory(start_position, goal, start_time, end_time, time[i])
    
    J = my_robot.get_jacobian(q, 'O')

    v_des = (J[3:] @ trajectory_velocity)+ K_P * (my_robot.FK(trajectory_position)[:3,3]-end_effector_position)
    joint_velocity_des = np.linalg.pinv(J[3:]) @ v_des
    joint_torques = K_D * (joint_velocity_des - dq)
    simulator.send_joint_torque((joint_torques+my_robot.g(q)))
    simulator.step()